In [1]:
import pandas as pd
import numpy as np
import calendar
import timeit
import datetime as dt
import time
import warnings
warnings.filterwarnings("ignore")
import math

## Deal with the data type for each column

In [2]:
df = pd.read_csv('card transactions.csv')
df.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2006-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2006-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2006-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [3]:
df.shape

(96753, 10)

In [4]:
df.dtypes

Recnum                 int64
Cardnum                int64
Date                  object
Merchnum              object
Merch description     object
Merch state           object
Merch zip            float64
Transtype             object
Amount               float64
Fraud                  int64
dtype: object

In [5]:
df['Date'] = pd.to_datetime(df['Date'])

In [6]:
df['Merch zip'].isnull().sum()

4656

In [7]:
new_zip = []
for i in df['Merch zip']:
    if pd.isnull(i) == True:
        new_zip.append(i)
    else:
        new_zip.append('{0:0>5}'.format(str(int(i))))

In [8]:
df['Merch zip'] = new_zip

In [9]:
df.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0
1,2,5142183973,2006-01-01,61003026333,SERVICE MERCHANDISE #81,MA,01803,P,31.42,0
2,3,5142131721,2006-01-01,4503082993600,OFFICE DEPOT #191,MD,20706,P,178.49,0
3,4,5142148452,2006-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118,P,3.62,0
4,5,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0


## Clean Data

### delete some transactions

In [10]:
df = df[df['Transtype']=='P']

In [11]:
df['Amount'].sort_values().tail(5)

89673      27218.00
80886      28392.84
59516      30372.46
47339      47900.00
52714    3102045.53
Name: Amount, dtype: float64

In [12]:
df = df[df['Amount']!=3102045.53]

In [13]:
df.isnull().sum()

Recnum                  0
Cardnum                 0
Date                    0
Merchnum             3198
Merch description       0
Merch state          1020
Merch zip            4300
Transtype               0
Amount                  0
Fraud                   0
dtype: int64

### Merch Num

In [14]:
df['Merchnum'].sort_values().head()

33056    0
20246    0
68476    0
36744    0
91980    0
Name: Merchnum, dtype: object

In [15]:
df.isnull().sum()

Recnum                  0
Cardnum                 0
Date                    0
Merchnum             3198
Merch description       0
Merch state          1020
Merch zip            4300
Transtype               0
Amount                  0
Fraud                   0
dtype: int64

In [16]:
merchdes_merchnum = {}
for index, merchdes in df[df['Merch description'].notnull()][df['Merchnum'].notnull()]['Merch description'].items():
    
    if pd.isnull(merchdes) == True:
        continue
    elif merchdes not in merchdes_merchnum :
        merchdes_merchnum[merchdes] = df.loc[index, 'Merchnum']

# fill in by mapping with Merch description
df['Merchnum'] = df['Merchnum'].fillna(df['Merch description'].map(merchdes_merchnum))

print(df.isnull().sum())

# assign unknown for adjustments transactions
df['Merchnum'] = df['Merchnum'].mask(df['Merch description'] == 'RETAIL CREDIT ADJUSTMENT', 'unknown')
df['Merchnum'] = df['Merchnum'].mask(df['Merch description'] == 'RETAIL DEBIT ADJUSTMENT', 'unknown')


Recnum                  0
Cardnum                 0
Date                    0
Merchnum             2038
Merch description       0
Merch state          1020
Merch zip            4300
Transtype               0
Amount                  0
Fraud                   0
dtype: int64


In [17]:
replace_0 = []
for i in df['Merchnum']:
    if i == '0':
        replace_0.append(np.NaN)
    elif pd.isnull(i) == True:
        replace_0.append(i)
    elif type(i) == str:
        replace_0.append(i.replace(" ",""))
    else:
        replace_0.append(str(int(i)).replace(" ",""))
df['Merchnum'] = replace_0

In [18]:
df.isnull().sum()

Recnum                  0
Cardnum                 0
Date                    0
Merchnum             1403
Merch description       0
Merch state          1020
Merch zip            4300
Transtype               0
Amount                  0
Fraud                   0
dtype: int64

### Merch zip

In [19]:
merchnum_zip = {}
for index, merchnum in df[df['Merchnum'].notnull()][df['Merch zip'].notnull()]['Merchnum'].items():
    if merchnum not in merchnum_zip :
        merchnum_zip [merchnum] = df.loc[index, 'Merch zip']

merchdes_zip = {}
for index, merchdes in df[df['Merch description'].notnull()][df['Merch zip'].notnull()]['Merch description'].items():
    if merchdes not in merchdes_zip :
        merchdes_zip [merchdes] = df.loc[index, 'Merch zip']

# fill in by mapping with merchnum and merch description
df['Merch zip'] = df['Merch zip'].fillna(df['Merchnum'].map(merchnum_zip))
df['Merch zip'] = df['Merch zip'].fillna(df['Merch description'].map(merchdes_zip))

# assign unknown for adjustments transactions
df['Merch zip'] = df['Merch zip'].mask(df['Merch description'] == 'RETAIL CREDIT ADJUSTMENT', 'unknown')
df['Merch zip'] = df['Merch zip'].mask(df['Merch description'] == 'RETAIL DEBIT ADJUSTMENT', 'unknown')



In [20]:
df.isnull().sum()

Recnum                  0
Cardnum                 0
Date                    0
Merchnum             1403
Merch description       0
Merch state          1020
Merch zip            1723
Transtype               0
Amount                  0
Fraud                   0
dtype: int64

### Merch state

In [21]:
# dict for mapping
zip_state = {}
for index, zip5 in df[df['Merch zip'].notnull()][df['Merch state'].notnull()]['Merch zip'].items():
    if zip5 not in zip_state:
        zip_state[zip5] = df.loc[index, 'Merch state']

for i in range(600,800):
    zipcode = '{0:0>5}'.format(str(int(i)))
    zip_state[zipcode] = 'PR'

for i in range(900,1000):
    zipcode = '{0:0>5}'.format(str(int(i)))
    zip_state[zipcode] = 'PR'

zip_state['00926'] = 'PR'
zip_state['00929'] = 'PR'
zip_state['00934'] = 'PR'
zip_state['00902'] = 'PR'
zip_state['00738'] = 'PR'
zip_state['90805'] = 'CA'
zip_state['76302'] = 'TX'
zip_state['00914'] = 'PR'
zip_state['95461'] = 'CA'
zip_state['00680'] = 'PR'
zip_state['00623'] = 'PR'
zip_state['00726'] = 'PR'
zip_state['00936'] = 'PR'
zip_state['12108'] = 'NY'
zip_state['00791'] = 'PR'
zip_state['00907'] = 'PR'
zip_state['00922'] = 'PR'
zip_state['00920'] = 'PR'
zip_state['00801'] = 'VI'
zip_state['31040'] = 'GA'
zip_state['41160'] = 'KY'
zip_state['00681'] = 'PR'

merchnum_state = {}
for index, merchnum in df[df['Merchnum'].notnull()][df['Merch state'].notnull()]['Merchnum'].items():
    if merchnum not in merchnum_state :
        merchnum_state [merchnum] = df.loc[index, 'Merch state']

merchdes_state = {}
for index, merchdes in df[df['Merch description'].notnull()][df['Merch state'].notnull()]['Merch description'].items():
    if merchdes not in merchdes_state :
        merchdes_state [merchdes] = df.loc[index, 'Merch state']

# fill in by mapping with zip, merchnum and merch description
df['Merch state'] = df['Merch state'].fillna(df['Merch zip'].map(zip_state))
df['Merch state'] = df['Merch state'].fillna(df['Merchnum'].map(merchnum_state))
df['Merch state'] = df['Merch state'].fillna(df['Merch description'].map(merchdes_state))

# assign unknown for adjustments transactions
df['Merch state'] = df['Merch state'].mask(df['Merch description'] == 'RETAIL CREDIT ADJUSTMENT', 'unknown')
df['Merch state'] = df['Merch state'].mask(df['Merch description'] == 'RETAIL DEBIT ADJUSTMENT', 'unknown')


In [22]:
df.isnull().sum()

Recnum                  0
Cardnum                 0
Date                    0
Merchnum             1403
Merch description       0
Merch state           283
Merch zip            1723
Transtype               0
Amount                  0
Fraud                   0
dtype: int64

In [23]:
# fill the rest with unknown
df = df.fillna('unknown')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96397 entries, 0 to 96752
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Recnum             96397 non-null  int64         
 1   Cardnum            96397 non-null  int64         
 2   Date               96397 non-null  datetime64[ns]
 3   Merchnum           96397 non-null  object        
 4   Merch description  96397 non-null  object        
 5   Merch state        96397 non-null  object        
 6   Merch zip          96397 non-null  object        
 7   Transtype          96397 non-null  object        
 8   Amount             96397 non-null  float64       
 9   Fraud              96397 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 10.1+ MB


In [24]:
df[df['Merchnum']=='unknown'].shape[0]

2094

In [25]:
df[df['Merch state']=='unknown'].shape[0]

974

In [26]:
df[df['Merch zip']=='unknown'].shape[0]

2414

In [27]:
df[df['Merch description']=='RETAIL DEBIT ADJUSTMENT'].shape[0]

308

## Create Variables

In [28]:
df.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0
1,2,5142183973,2006-01-01,61003026333,SERVICE MERCHANDISE #81,MA,01803,P,31.42,0
2,3,5142131721,2006-01-01,4503082993600,OFFICE DEPOT #191,MD,20706,P,178.49,0
3,4,5142148452,2006-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118,P,3.62,0
4,5,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0


### DOW TE

In [29]:
# make day of week targeting
df['dow'] = df.Date.apply(lambda x: calendar.day_name[x.weekday()])
train_test = df[df.Date <'2006-11-01']

In [30]:
# statistical smoothing
c = 4
nmid = 20
y_avg = train_test['Fraud'].mean()
y_dow = train_test.groupby('dow')['Fraud'].mean()
num = train_test.groupby('dow').size()
y_dow_smooth = y_avg + (y_dow -y_avg)/(1+np.exp(-(num-nmid)/c))
df['dow_risk'] = df.dow.map(y_dow_smooth)

In [31]:
y_dow

dow
Friday       0.009788
Monday       0.010095
Saturday     0.018626
Sunday       0.025994
Thursday     0.007127
Tuesday      0.009630
Wednesday    0.008711
Name: Fraud, dtype: float64

In [32]:
y_dow_smooth

dow
Friday       0.009788
Monday       0.010095
Saturday     0.018626
Sunday       0.025994
Thursday     0.007127
Tuesday      0.009630
Wednesday    0.008711
dtype: float64

### Amount cat

In [33]:
df['Amount'].quantile([0.1*i for i in range(0,10)])

0.0       0.010
0.1       4.370
0.2      21.740
0.3      48.000
0.4      85.000
0.5     137.980
0.6     216.000
0.7     339.942
0.8     550.570
0.9    1088.000
Name: Amount, dtype: float64

In [34]:
df['Amount'].describe()

count    96397.000000
mean       395.858494
std        832.330197
min          0.010000
25%         33.460000
50%        137.980000
75%        428.250000
max      47900.000000
Name: Amount, dtype: float64

In [35]:
amount_type = []
for i in df['Amount']:
    if i <= 5:
        amount_type.append('extremely small amount')
    elif i <= 20:
        amount_type.append('small amount')
    elif i <= 50:
        amount_type.append('normal amount')
    elif i <= 150:
        amount_type.append('relative large amount')
    elif i <= 500:
        amount_type.append('large amount')
    else:
        amount_type.append('extremely large amount')
df['Amount_type'] = amount_type

In [36]:
# statistical smoothing
train_test = df[df.Date <'2006-11-01']
c = 4
nmid = 20
y_avg = train_test['Fraud'].mean()
y_atype = train_test.groupby('Amount_type')['Fraud'].mean()
num = train_test.groupby('Amount_type').size()
y_atype_smooth = y_avg + (y_atype -y_avg)/(1+np.exp(-(num-nmid)/c))
df['atype_risk'] = df.Amount_type.map(y_atype_smooth)

In [37]:
y_atype

Amount_type
extremely large amount    0.036103
extremely small amount    0.000214
large amount              0.005327
normal amount             0.005832
relative large amount     0.002227
small amount              0.000533
Name: Fraud, dtype: float64

In [38]:
y_atype_smooth

Amount_type
extremely large amount    0.036103
extremely small amount    0.000214
large amount              0.005327
normal amount             0.005832
relative large amount     0.002227
small amount              0.000533
dtype: float64

### linking variables

In [39]:
df.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,dow,dow_risk,Amount_type,atype_risk
0,1,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,Sunday,0.025994,extremely small amount,0.000214
1,2,5142183973,2006-01-01,61003026333,SERVICE MERCHANDISE #81,MA,01803,P,31.42,0,Sunday,0.025994,normal amount,0.005832
2,3,5142131721,2006-01-01,4503082993600,OFFICE DEPOT #191,MD,20706,P,178.49,0,Sunday,0.025994,large amount,0.005327
3,4,5142148452,2006-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118,P,3.62,0,Sunday,0.025994,extremely small amount,0.000214
4,5,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,Sunday,0.025994,extremely small amount,0.000214


In [40]:
df.dtypes

Recnum                        int64
Cardnum                       int64
Date                 datetime64[ns]
Merchnum                     object
Merch description            object
Merch state                  object
Merch zip                    object
Transtype                    object
Amount                      float64
Fraud                         int64
dow                          object
dow_risk                    float64
Amount_type                  object
atype_risk                  float64
dtype: object

In [41]:
df['Cardnum'] = df['Cardnum'].astype(str)

In [42]:
df.dtypes

Recnum                        int64
Cardnum                      object
Date                 datetime64[ns]
Merchnum                     object
Merch description            object
Merch state                  object
Merch zip                    object
Transtype                    object
Amount                      float64
Fraud                         int64
dow                          object
dow_risk                    float64
Amount_type                  object
atype_risk                  float64
dtype: object

In [43]:
df['Cardnum_Merchnum'] = df['Cardnum']+df['Merchnum']
df['Cardnum_Merstate'] = df['Cardnum']+df['Merch state']
df['Cardnum_Merzip'] = df['Cardnum']+df['Merch zip']
df['Cardnum_Atype'] = df['Cardnum']+df['Amount_type']
df['Merchnum_Atype'] = df['Merchnum']+df['Amount_type']
df['State_Atype'] = df['Merch state']+df['Amount_type']
df['Zip_Atype'] = df['Merch zip']+df['Amount_type']

In [44]:
df.columns

Index(['Recnum', 'Cardnum', 'Date', 'Merchnum', 'Merch description',
       'Merch state', 'Merch zip', 'Transtype', 'Amount', 'Fraud', 'dow',
       'dow_risk', 'Amount_type', 'atype_risk', 'Cardnum_Merchnum',
       'Cardnum_Merstate', 'Cardnum_Merzip', 'Cardnum_Atype', 'Merchnum_Atype',
       'State_Atype', 'Zip_Atype'],
      dtype='object')

In [45]:
entities = ['Cardnum', 'Merchnum', 'Cardnum_Merchnum','Cardnum_Merstate', 'Cardnum_Merzip', 'Cardnum_Atype', 'Merchnum_Atype',
       'State_Atype', 'Zip_Atype']

### Basic Calculation of Vars

In [46]:
df1=df.copy()
df1['check_date']=df1.Date
df1['check_record']=df1.Recnum
vars = df.copy()

In [47]:
one_year = dt.timedelta(days=365)

In [48]:
start = timeit.default_timer()
for entity in entities:    
    try: 
        print('Run time for the last entity ----------------- {}s'.format(timeit.default_timer() - st))
    except: 
        print('')
    st = timeit.default_timer()    
    ## Day-since variables:   
    df_l = df1[['Recnum', 'Date', entity]]
    df_r = df1[['check_record', 'check_date', entity, 'Amount']]    
    temp = pd.merge(df_l, df_r, left_on = entity, right_on = entity)   
    temp1 = temp[temp.Recnum > temp.check_record][['Recnum','Date','check_date']]\
                                                   .groupby('Recnum')[['Date', 'check_date']].last()
    mapper = (temp1.Date - temp1.check_date).dt.days
    vars[entity + '_day_since'] = vars.Recnum.map(mapper)
    vars[entity + '_day_since'].fillna(one_year, inplace = True)
    print('\n' + entity + '_day_since ---> Done')   
    ## Frequency & Amount variables:   
    for time in [0,1,3,7,14,30]:       
        temp2 = temp[(temp.check_date >= (temp.Date - dt.timedelta(time))) &\
                       (temp.Recnum >= temp.check_record)][['Recnum', entity, 'Amount']]       
        col_name = entity + '_count_' + str(time)    
        mapper2 = temp2.groupby('Recnum')[entity].count()      
        vars[col_name] = vars.Recnum.map(mapper2)        
        print(col_name + ' ---> Done')       
        vars[entity + '_avg_' + str(time)] = vars.Recnum.map(temp2.groupby('Recnum')['Amount'].mean())
        vars[entity + '_max_' + str(time)] = vars.Recnum.map(temp2.groupby('Recnum')['Amount'].max())
        vars[entity + '_med_' + str(time)] = vars.Recnum.map(temp2.groupby('Recnum')['Amount'].median())
        vars[entity + '_total_' + str(time)] = vars.Recnum.map(temp2.groupby('Recnum')['Amount'].sum())
        vars[entity + '_actual/avg_' + str(time)] = vars['Amount'] / vars[entity + '_avg_' + str(time)]
        vars[entity + '_actual/max_' + str(time)] = vars['Amount'] / vars[entity + '_max_' + str(time)]
        vars[entity + '_actual/med_' + str(time)] = vars['Amount'] / vars[entity + '_med_' + str(time)]
        vars[entity + '_actual/toal_' + str(time)] = vars['Amount'] / vars[entity + '_total_' + str(time)]       
        print(entity + ' amount variables over past ' + str(time) + ' ---> Done')

print('Total run time: {}mins'.format((timeit.default_timer() - start)/60))




Cardnum_day_since ---> Done
Cardnum_count_0 ---> Done
Cardnum amount variables over past 0 ---> Done
Cardnum_count_1 ---> Done
Cardnum amount variables over past 1 ---> Done
Cardnum_count_3 ---> Done
Cardnum amount variables over past 3 ---> Done
Cardnum_count_7 ---> Done
Cardnum amount variables over past 7 ---> Done
Cardnum_count_14 ---> Done
Cardnum amount variables over past 14 ---> Done
Cardnum_count_30 ---> Done
Cardnum amount variables over past 30 ---> Done
Run time for the last entity ----------------- 7.502459163999987s

Merchnum_day_since ---> Done
Merchnum_count_0 ---> Done
Merchnum amount variables over past 0 ---> Done
Merchnum_count_1 ---> Done
Merchnum amount variables over past 1 ---> Done
Merchnum_count_3 ---> Done
Merchnum amount variables over past 3 ---> Done
Merchnum_count_7 ---> Done
Merchnum amount variables over past 7 ---> Done
Merchnum_count_14 ---> Done
Merchnum amount variables over past 14 ---> Done
Merchnum_count_30 ---> Done
Merchnum amount variables o

In [49]:
vars.shape

(96397, 516)

In [50]:
start = timeit.default_timer()
# velocity change variables
for ent in entities:
    for d in ['0', '1']:
        for dd in ['3','7', '14', '30']:
            vars[ent + '_count_' + d + '_by_' + dd] =\
            vars[ent + '_count_' + d]/(vars[ent + '_count_' + dd])/float(dd)
            
print('run time: {}s'.format(timeit.default_timer() - start))


run time: 0.5429943579999872s


In [51]:
# maximum indicator
for i in entities:
    for v in ['3','7','30']:
        mappers=vars.groupby(i)[i+'_count_'+v].max()
        vars['max_count_by_'+i+'_'+v]=vars[i].map(mappers)

In [52]:
# relative velocity change variables
for entity in entities:
    for field in entities:
        if field==entity:
            continue
        else:
            df_c=df1[['Recnum','Date',entity]]
            df_d=df1[['check_record','check_date',entity,field]]
            temp=pd.merge(df_c, df_d, left_on=entity, right_on=entity)
        #number of unique for particular field
        for offset_t in [1,3,7,14,30,60]:
            count_day_df=temp[(temp.check_date>=(temp.Date-dt.timedelta(offset_t)))
                             &(temp.Recnum>=temp.check_record)]
            col_name=f'{entity}_unique_count_for_{field}_{offset_t}'
            mapper=count_day_df.groupby(['Recnum'])[field].nunique()
            vars[col_name]=vars.Recnum.map(mapper)
        print(f'Run time for entity {entity} in field {field}')

Run time for entity Cardnum in field Merchnum
Run time for entity Cardnum in field Cardnum_Merchnum
Run time for entity Cardnum in field Cardnum_Merstate
Run time for entity Cardnum in field Cardnum_Merzip
Run time for entity Cardnum in field Cardnum_Atype
Run time for entity Cardnum in field Merchnum_Atype
Run time for entity Cardnum in field State_Atype
Run time for entity Cardnum in field Zip_Atype
Run time for entity Merchnum in field Cardnum
Run time for entity Merchnum in field Cardnum_Merchnum
Run time for entity Merchnum in field Cardnum_Merstate
Run time for entity Merchnum in field Cardnum_Merzip
Run time for entity Merchnum in field Cardnum_Atype
Run time for entity Merchnum in field Merchnum_Atype
Run time for entity Merchnum in field State_Atype
Run time for entity Merchnum in field Zip_Atype
Run time for entity Cardnum_Merchnum in field Cardnum
Run time for entity Cardnum_Merchnum in field Merchnum
Run time for entity Cardnum_Merchnum in field Cardnum_Merstate
Run time fo

In [53]:
vars.shape

(96397, 1047)

### Benford's Law Variables

In [54]:
cleaned_data = df[df['Merch description'].str.match(r"[Ff][Ee][Dd][Ee][Xx]")==False].copy()
cleaned_data['Amount'] = cleaned_data['Amount'].astype(str)

In [55]:
def n_low(df):
    return ((df['Amount'].str.extract(r"([1-9])")=='1') | (df['Amount'].str.extract(r"([1-9])")=='2')).sum()

In [56]:
cardnums = cleaned_data.groupby('Cardnum').apply(n_low)
cardnums['n'] = cleaned_data.groupby('Cardnum')['Amount'].count()
cardnums['n_low'] = cleaned_data.groupby('Cardnum').apply(n_low)
cardnums = cardnums[['n','n_low']]
cardnums['n_high'] = cardnums['n'] - cardnums['n_low']

In [57]:
cardnums['n_low'] = cardnums['n_low'].replace(0,1)
cardnums['n_high'] = cardnums['n_high'].replace(0,1)

In [58]:
cardnums['R'] = 1.096 * cardnums['n_low']/cardnums['n_high']
cardnums['1/R'] = 1/ cardnums['R']
cardnums['U'] = cardnums[['R','1/R']].max(axis=1)

In [59]:
cardnums['t'] = (cardnums['n']-15)/3
cardnums['U*'] = 1+((cardnums['U']-1)/(1+np.exp(-cardnums['t'])))

In [60]:
cardnums.sort_values('U*',ascending=False)[0:40].to_csv('Top 40 Cardnum.csv')

In [61]:
merchnums = cleaned_data.groupby('Merchnum').apply(n_low)
merchnums['n'] = cleaned_data.groupby('Merchnum')['Amount'].count()
merchnums['n_low'] = cleaned_data.groupby('Merchnum').apply(n_low)
merchnums = cardnums[['n','n_low']]
merchnums['n_high'] = merchnums['n'] - merchnums['n_low']

In [62]:
merchnums['n_low'] = merchnums['n_low'].replace(0,1)
merchnums['n_high'] = merchnums['n_high'].replace(0,1)

In [63]:
merchnums['R'] = 1.096 * merchnums['n_low']/merchnums['n_high']
merchnums['1/R'] = 1/ merchnums['R']
merchnums['U'] = merchnums[['R','1/R']].max(axis=1)

In [64]:
merchnums['t'] = (merchnums['n']-15)/3
merchnums['U*'] = 1+((merchnums['U']-1)/(1+np.exp(-merchnums['t'])))

In [65]:
merchnums.sort_values('U*',ascending=False)[0:40].to_csv('Top 40 Merchnum.csv')

In [66]:
cardnums.shape

(1640, 8)

In [67]:
df.shape

(96397, 21)

In [68]:
cleaned_data.shape

(84622, 21)

In [69]:
cardnums.head()

,n,n_low,n_high,R,1/R,U,t,U*
Cardnum,,,,,,,,
5142110002,1,1,1,1.096,0.912409,1.096000,-4.666667,1.000894
5142110081,4,1,4,0.274,3.649635,3.649635,-3.666667,1.066041
5142110313,3,2,1,2.192,0.456204,2.192000,-4.000000,1.021440
5142110402,11,3,8,0.411,2.433090,2.433090,-1.333333,1.298955
5142110434,1,1,1,1.096,0.912409,1.096000,-4.666667,1.000894


In [87]:
vars['cardnums_b'] = vars.Cardnum.map(cardnums['U*'])

In [88]:
vars['merchnums_b'] = vars.Merchnum.map(merchnums['U*'])

In [89]:
vars.isnull().sum()

Recnum                                           0
Cardnum                                          0
Date                                             0
Merchnum                                         0
Merch description                                0
                                             ...  
Zip_Atype_unique_count_for_State_Atype_14        0
Zip_Atype_unique_count_for_State_Atype_30        0
Zip_Atype_unique_count_for_State_Atype_60        0
cardnums_b                                      72
merchnums_b                                  96397
Length: 1049, dtype: int64

In [90]:
vars.fillna(0)

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,...,Zip_Atype_unique_count_for_Merchnum_Atype_30,Zip_Atype_unique_count_for_Merchnum_Atype_60,Zip_Atype_unique_count_for_State_Atype_1,Zip_Atype_unique_count_for_State_Atype_3,Zip_Atype_unique_count_for_State_Atype_7,Zip_Atype_unique_count_for_State_Atype_14,Zip_Atype_unique_count_for_State_Atype_30,Zip_Atype_unique_count_for_State_Atype_60,cardnums_b,merchnums_b
0,1,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,...,1,1,1,1,1,1,1,1,2.178008,0.0
1,2,5142183973,2006-01-01,61003026333,SERVICE MERCHANDISE #81,MA,01803,P,31.42,0,...,1,1,1,1,1,1,1,1,1.604857,0.0
2,3,5142131721,2006-01-01,4503082993600,OFFICE DEPOT #191,MD,20706,P,178.49,0,...,1,1,1,1,1,1,1,1,2.368143,0.0
3,4,5142148452,2006-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118,P,3.62,0,...,1,1,1,1,1,1,1,1,1.044105,0.0
4,5,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,...,1,1,1,1,1,1,1,1,2.178008,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96748,96749,5142276053,2006-12-31,3500000006160,BEST BUY 00001610,KY,41042,P,84.79,0,...,2,2,1,1,1,1,1,1,1.002393,0.0
96749,96750,5142225701,2006-12-31,8090710030950,MARKUS OFFICE SUPPLIES,OH,45248,P,118.75,0,...,1,1,1,1,1,1,1,1,1.137948,0.0
96750,96751,5142226486,2006-12-31,4503057341100,"TECH PAC, INC",OH,45150,P,363.56,0,...,1,1,1,1,1,1,1,1,1.201338,0.0
96751,96752,5142244619,2006-12-31,8834000695412,BUY.COM,CA,92656,P,2202.03,0,...,1,1,1,1,1,1,1,1,1.499767,0.0


In [77]:
vars.shape

(96397, 1049)

In [91]:
vars.to_csv('project3vars.csv')